# Binning

In [46]:
# we will be studying to numerical data to categorical data

# we have two techniques to do so
# the first technique is Discritizatioin (binning)
# the second technique is Binarization


## Discretization

In [47]:

"""  

# its the processs of transforming continuous variables into discrete variables by creating a set of contiguous intervals that span the range of the variable's values. It's also called  binning, where bin is an alternative name for interval.

# we use it to handleo utliers, and to improve the value spread


Three types of binninng
1/ unsupervised binning
    i/ equal width binning
        - here you put all the numerical values into bins
        - useful for handling outlieres
        - no changes btwn original data histogram and after binning data histogram while plotting it

    ii/ equal frequency binning / quantile binning
        - suppose you want 10 bins on the data, then you must make sure that each interval must contain 10% of total observaions
        - here the width of the intervals may not be equal
        - useful for handling ourliers
        - makes the values spread uniform (all histogram bars will be equal)
        - HIGHLY USED

    iii/ k means binning
        - here you use a clustering algorithm called k means
        - mostly used when the data is divided into many clusters
        - here intervals are called as centroids
        - suppose you want 5 intervals


2/ supervised binning
    i/ decision tree binning
3/ custom binning




"""




"  \n\n# its the processs of transforming continuous variables into discrete variables by creating a set of contiguous intervals that span the range of the variable's values. It's also called  binning, where bin is an alternative name for interval.\n\n# we use it to handleo utliers, and to improve the value spread\n\n\nThree types of binninng\n1/ unsupervised binning\n    i/ equal width binning\n        - here you put all the numerical values into bins\n        - useful for handling outlieres\n        - no changes btwn original data histogram and after binning data histogram while plotting it\n\n    ii/ equal frequency binning / quantile binning\n        - suppose you want 10 bins on the data, then you must make sure that each interval must contain 10% of total observaions\n        - here the width of the intervals may not be equal\n        - useful for handling ourliers\n        - makes the values spread uniform (all histogram bars will be equal)\n        - HIGHLY USED\n\n    iii/ k m

## K means binning

In [48]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [49]:
df = pd.read_csv('train.csv', usecols=['Age', 'Fare', 'Survived'])
df = df.dropna()
print(df.shape)
df.head(3)

(714, 3)


,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250


In [50]:
x = df.drop(columns='Survived')
y = df['Survived']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier()

clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print(f"Accuracy : {np.round(accuracy_score(y_test, y_pred),2)*100} %")
print(f"Cross val : {np.round(np.mean(cross_val_score(clf, x_train, y_train, cv=5, scoring='accuracy')),2)*100} %")


Accuracy : 63.0 %
Cross val : 64.0 %


applying k means discritizer

In [ ]:
# applying discritizer

kbin_age = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')   
# other things you can put instead of 'quantile' ----- 'uniform' (equal-width), 'quantile' (equal-frequency), 'kmeans' (clustering-based)
kbin_fare = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')

trf = ColumnTransformer([
    ('first', kbin_age, [0]),       # will take the first column passed in the dataset
    ('second', kbin_fare, [1])      # will take the second column passed in the dataset
])

x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

print(f"checking number of bins : {trf.named_transformers_['first'].n_bins_}")
print(f"checking edges of bins : \n{trf.named_transformers_['first'].bin_edges_}\n")

output = pd.DataFrame({
    'age':x_train['Age'],
    'age_trf': x_train_trf[:, 0],
    'fare': x_train['Fare'],
    'fare_trf': x_train_trf[:,1]
})

output


checking number of bins : [10]
checking edges of bins : 
[array([ 0.42, 14.  , 19.  , 22.  , 25.  , 28.5 , 32.  , 36.  , 42.  ,
        50.  , 80.  ])                                                ]



C:\Users\biswa\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
C:\Users\biswa\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


,age,age_trf,fare,fare_trf
328,31.0,5.0,20.5250,5.0
73,26.0,4.0,14.4542,4.0
253,30.0,5.0,16.1000,5.0
719,33.0,6.0,7.7750,1.0
666,25.0,4.0,13.0000,4.0
...,...,...,...,...
92,46.0,8.0,61.1750,8.0
134,25.0,4.0,13.0000,4.0
337,41.0,7.0,134.5000,9.0
548,33.0,6.0,20.5250,5.0


In [68]:
output['age_labels'] = pd.cut(df['Age'], bins=trf.named_transformers_['first'].bin_edges_[0].tolist())

output['fare_labels'] = pd.cut(df['Fare'], bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

output

,age,age_trf,fare,fare_trf,age_labels,fare_labels
328,31.0,5.0,20.5250,5.0,"(28.5, 32.0]","(15.75, 26.0]"
73,26.0,4.0,14.4542,4.0,"(25.0, 28.5]","(13.0, 15.75]"
253,30.0,5.0,16.1000,5.0,"(28.5, 32.0]","(15.75, 26.0]"
719,33.0,6.0,7.7750,1.0,"(32.0, 36.0]","(7.75, 7.896]"
666,25.0,4.0,13.0000,4.0,"(22.0, 25.0]","(9.225, 13.0]"
...,...,...,...,...,...,...
92,46.0,8.0,61.1750,8.0,"(42.0, 50.0]","(51.479, 82.171]"
134,25.0,4.0,13.0000,4.0,"(22.0, 25.0]","(9.225, 13.0]"
337,41.0,7.0,134.5000,9.0,"(36.0, 42.0]","(82.171, 512.329]"
548,33.0,6.0,20.5250,5.0,"(32.0, 36.0]","(15.75, 26.0]"
